In [1]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

from ecmwfapi import ECMWFDataServer
import os
import sys
import dask
import numpy as np
from esio import download
import datetime
from esio import EsioData as ed
import pandas as pd

import calendar
from datetime import datetime
import os
from ecmwfapi import ECMWFDataServer

'''
Download models with sea ice forecasts within the 2s2 forecast data archive.

https://software.ecmwf.int/wiki/display/S2S/Models

'''

'\nDownload models with sea ice forecasts within the 2s2 forecast data archive.\n\nhttps://software.ecmwf.int/wiki/display/S2S/Models\n\n'

In [2]:
# ecmwf makes a hindcast everytime they make a new forecast
# they label these hindcast by the Model Version Date
# which oddly is not the current date but rather a few weeks ahead
# Then for eace of these versions, they run a forecast for the same mon and day
# from 1999 to one year ago

most_recent = np.datetime64('today')+ np.timedelta64(14,'D')
year_ago= np.datetime64('today') - np.timedelta64(351,'D')
versions = np.arange(year_ago, most_recent)
print(len(versions))
versions = versions[np.is_busday(versions,weekmask = "Mon Thu")]
print(versions)

365
['2018-07-12' '2018-07-16' '2018-07-19' '2018-07-23' '2018-07-26'
 '2018-07-30' '2018-08-02' '2018-08-06' '2018-08-09' '2018-08-13'
 '2018-08-16' '2018-08-20' '2018-08-23' '2018-08-27' '2018-08-30'
 '2018-09-03' '2018-09-06' '2018-09-10' '2018-09-13' '2018-09-17'
 '2018-09-20' '2018-09-24' '2018-09-27' '2018-10-01' '2018-10-04'
 '2018-10-08' '2018-10-11' '2018-10-15' '2018-10-18' '2018-10-22'
 '2018-10-25' '2018-10-29' '2018-11-01' '2018-11-05' '2018-11-08'
 '2018-11-12' '2018-11-15' '2018-11-19' '2018-11-22' '2018-11-26'
 '2018-11-29' '2018-12-03' '2018-12-06' '2018-12-10' '2018-12-13'
 '2018-12-17' '2018-12-20' '2018-12-24' '2018-12-27' '2018-12-31'
 '2019-01-03' '2019-01-07' '2019-01-10' '2019-01-14' '2019-01-17'
 '2019-01-21' '2019-01-24' '2019-01-28' '2019-01-31' '2019-02-04'
 '2019-02-07' '2019-02-11' '2019-02-14' '2019-02-18' '2019-02-21'
 '2019-02-25' '2019-02-28' '2019-03-04' '2019-03-07' '2019-03-11'
 '2019-03-14' '2019-03-18' '2019-03-21' '2019-03-25' '2019-03-28'
 '2019

In [13]:
# test my understanding of date/time
vdates = pd.DatetimeIndex(versions)

#for ct in np.arange(0,len(vdates),1):
for ct in [100]:
    year = vdates[ct].year
    month = vdates[ct].month
    day = vdates[ct].day
    DS = np.datetime64(datetime(1999, month, day, 0, 0))
    DE = np.datetime64(datetime(year-1, month, day, 0, 0))

    hdates = pd.date_range(start=DS,end=DE,freq=pd.DateOffset(years=1))
    hdates = [x.strftime('%Y-%m-%d') for x in hdates]
    print(vdates[ct].strftime('%Y-%m-%d'))
    print(hdates)

2019 6 27
2019-06-27
['1999-06-27', '2000-06-27', '2001-06-27', '2002-06-27', '2003-06-27', '2004-06-27', '2005-06-27', '2006-06-27', '2007-06-27', '2008-06-27', '2009-06-27', '2010-06-27', '2011-06-27', '2012-06-27', '2013-06-27', '2014-06-27', '2015-06-27', '2016-06-27', '2017-06-27', '2018-06-27']


In [14]:
E = ed.EsioData.load()
main_dir = E.model_dir

In [15]:
# a starting point
mod_dicts = {}

# # ECMWF
mod_dicts['ecmwf'] = {
 "class": "s2",
 "dataset": "s2s",
 "expver": "prod",
 "date": "2019-07-11",                                                               
 "hdate": "2005-07-11",                                                          
 "levtype": "sfc",
 "model": "glob",
 "origin": "ecmf",
 "param": "31",
 "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104",
 "stream": "enfh",
 "time": "00:00:00",
 "type": "cf",
 "target": "output",
 }


In [16]:
#@dask.delayed

def download_month(config_dict):
    # Start server
    cserver = ECMWFDataServer()
    cserver.retrieve(config_dict)
    return 1

In [ ]:
dataclass='s2s'
run_type='reforecast'
cmod = 'ecmwf'

vdates = pd.DatetimeIndex(versions)

for ct in np.arange(0,len(vdates),1):
    X = 1

    year = vdates[ct].year
    month = vdates[ct].month
    day = vdates[ct].day
    print(year, month, day)
    DS = np.datetime64(datetime(1999, month, day, 0, 0))
    DE = np.datetime64(datetime(year-1, month, day, 0, 0))

    hdates = pd.date_range(start=DS,end=DE,freq=pd.DateOffset(years=1))
    hdates = [x.strftime('%Y-%m-%d') for x in hdates]
    vdate = vdates[ct].strftime('%Y-%m-%d')
    print(hdates)
    
    for hd in np.arange(0,len(hdates),1):

        cdict = mod_dicts[cmod]
        cdict['date'] = vdate
        cdict['hdate'] = hdates[hd]

        target = os.path.join(main_dir, cmod, run_type, 'native',
                                  cmod+'_'+hdates[hd]+'.grib')
        cdict['target'] = target
        cdict['expect'] = 'any'

        print('Version Date is ',vdate,' hdate is ',hdates[hd])
        print('output file is ',target)
        print('/n/n')
        #X = download_month(cdict)
        #X.compute()
        X = X + download_month(cdict)

    # Call compute to download all years concurently for this date
    X.compute()

2018 7 19
['1999-07-19', '2000-07-19', '2001-07-19', '2002-07-19', '2003-07-19', '2004-07-19', '2005-07-19', '2006-07-19', '2007-07-19', '2008-07-19', '2009-07-19', '2010-07-19', '2011-07-19', '2012-07-19', '2013-07-19', '2014-07-19', '2015-07-19', '2016-07-19', '2017-07-19']
Version Date is  2018-07-19  hdate is  1999-07-19
output file is  /home/disk/sipn/nicway/data/model/ecmwf/reforecast/native/ecmwf_1999-07-19.grib
/n/n
Version Date is  2018-07-19  hdate is  2000-07-19
output file is  /home/disk/sipn/nicway/data/model/ecmwf/reforecast/native/ecmwf_2000-07-19.grib
/n/n
Version Date is  2018-07-19  hdate is  2001-07-19
output file is  /home/disk/sipn/nicway/data/model/ecmwf/reforecast/native/ecmwf_2001-07-19.grib
/n/n
Version Date is  2018-07-19  hdate is  2002-07-19
output file is  /home/disk/sipn/nicway/data/model/ecmwf/reforecast/native/ecmwf_2002-07-19.grib
/n/n
Version Date is  2018-07-19  hdate is  2003-07-19
output file is  /home/disk/sipn/nicway/data/model/ecmwf/reforecast/na

2019-06-27 22:31:47 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web-API+Troubleshooting or contact servicedesk@ecmwf.int. In case of delays, please check  service activity page https://apps.ecmwf.int/webapi-activity/
2019-06-27 22:31:47 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web-API+Troubleshooting or contact servicedesk@ecmwf.int. In case of delays, please check  service activity page https://apps.ecmwf.int/webapi-activity/
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 second